In [1]:
#importing libraries
import requests
import re
from bs4 import BeautifulSoup

In [3]:
link = "https://www.transfermarkt.de/premier-league/transfers/wettbewerb/GB1/plus/?saison_id=2023&s_w=&leihe=1&intern=0&intern=1"

response = requests.get(link, headers={"User-Agent": "Custom"})
print(response.text)

<!DOCTYPE html>
<html lang="de">

<head>
    
<script type="text/javascript" data-description="sourcepoint stub code">
    !function () { var e = function () { var e, t = "__tcfapiLocator", a = [], n = window; for (; n;) { try { if (n.frames[t]) { e = n; break } } catch (e) { } if (n === window.top) break; n = n.parent } e || (!function e() { var a = n.document, r = !!n.frames[t]; if (!r) if (a.body) { var i = a.createElement("iframe"); i.style.cssText = "display:none", i.name = t, a.body.appendChild(i) } else setTimeout(e, 5); return !r }(), n.__tcfapi = function () { for (var e, t = arguments.length, n = new Array(t), r = 0; r < t; r++)n[r] = arguments[r]; if (!n.length) return a; if ("setGdprApplies" === n[0]) n.length > 3 && 2 === parseInt(n[1], 10) && "boolean" == typeof n[3] && (e = n[3], "function" == typeof n[2] && n[2]("set", !0)); else if ("ping" === n[0]) { var i = { gdprApplies: e, cmpLoaded: !1, cmpStatus: "stub" }; "function" == typeof n[2] && n[2](i) } else a.push(n) }, 

In [4]:
##############################################################################
### 2.2 Parse HTML using BeautifulSoup & RegEx
##############################################################################
import pandas as pd

dftable = pd.DataFrame()
names = []
ages = []
nationalities = []
positions = []
values = []
previous_clubs = []
new_clubs = []
transfer_fees = []

soup = BeautifulSoup(response.content)

In [5]:
### 2.2.1 Use BeautifulSoup to identify all box class objects within the HTML
boxes = soup.find_all("div", {"class": "box"})
# The subpage contains 25 boxes (20 teams, 5 additional boxes)
# Search for "box" in the Firefox inspector tool.

### 2.2.2 Loop through each box
j = 0
for box in boxes:
    if (j > 2) & (
        j < (len(boxes) - 2)
    ):  # leave out the first 3 and last 2 boxes

        ### 2.2.3 Find the two table bodies (incoming & outgoing transfers)
        tbodies = box.find_all("tbody")
        # Distinguish between incoming & outgoing transfers (directional)
        incoming = tbodies[0]
        outgoing = tbodies[1]
        trs_incoming = incoming.find_all("tr")
        trs_outgoing = outgoing.find_all("tr")

        ### 2.2.4 Loop through each table row in a table body
        for tr in trs_incoming:
            new_club = re.findall('title="(.*?)">', str(box))[0]
            ### 2.2.5 Loop through each cell in a table row
            tds = tr.find_all("td")
            name = re.findall('title="(.*?)"', str(tds[0]))[0]
            age = re.findall('-cell">(.*?)<', str(tds[1]))[0]
            nationality = re.findall('title="(.*?)"', str(tds[2]))
            position = re.findall('cell">(.*?)<', str(tds[3]))[0]
            value = re.findall('cell">(.*?)<', str(tds[5]))[0]
            previous_club = re.findall('title="(.*?)"', str(tds[6]))[0]
            transfer_fee = re.findall(
                '[0-9][0-9][0-9][0-9]">(.*?)</a>', str(tds[8])
            )
            new_clubs.append(new_club)
            names.append(name)
            nationalities.append(nationality)
            positions.append(position)
            values.append(value)
            previous_clubs.append(previous_club)
            transfer_fees.append(transfer_fee)
            ages.append(age)

        ### 2.2.4 Loop through each table row in a table body
        for tr in trs_outgoing:
            previous_club = re.findall('title="(.*?)">', str(box))[0]
            ### 2.2.5 Loop through each cell in a table row
            tds = tr.find_all("td")
            name = re.findall('title="(.*?)"', str(tds[0]))[0]
            age = re.findall('-cell">(.*?)<', str(tds[1]))[0]
            nationality = re.findall('title="(.*?)"', str(tds[2]))
            position = re.findall('cell">(.*?)<', str(tds[3]))[0]
            value = re.findall('cell">(.*?)<', str(tds[5]))[0]
            new_club = re.findall('title="(.*?)"', str(tds[6]))[0]
            transfer_fee = re.findall(
                '[0-9][0-9][0-9][0-9]">(.*?)</a>', str(tds[8])
            )
            new_clubs.append(new_club)
            names.append(name)
            nationalities.append(nationality)
            positions.append(position)
            values.append(value)
            previous_clubs.append(previous_club)
            transfer_fees.append(transfer_fee)
            ages.append(age)
        j += 1
    else:
        print("Not relevant")
        j += 1

### 2.2.6 Consolidate in dataframe and save as csv
transfer_fees_adjusted = []
for transfer_fee in transfer_fees:
    transfer_fee = str(transfer_fee)
    if "Leih-Ende" in transfer_fee:
        transfer_fee = "Loan contract ended"
    elif "Leihgebühr" in transfer_fee:
        transfer_fee = transfer_fee.replace("Leihgebühr:", "Loan fee:")
        transfer_fee = transfer_fee.replace(
            '<br/><i class="normaler-text">', ""
        )
        transfer_fee = transfer_fee.replace("</i>", "")
    elif "Leihe" in transfer_fee:
        transfer_fee = transfer_fee.replace("Leihe", "Loan")
    elif "ablösefrei" in transfer_fee:
        transfer_fee = transfer_fee.replace("ablösefrei", "no fee")

    transfer_fee = transfer_fee.replace("['", "")
    transfer_fee = transfer_fee.replace("']", "")
    transfer_fees_adjusted.append(transfer_fee)

dftable["name"] = names
dftable["age"] = ages
dftable["nationality"] = nationalities
dftable["position"] = positions
dftable["value"] = values
dftable["previous_club"] = previous_clubs
dftable["new_club"] = new_clubs
dftable["transfer_fee"] = transfer_fees_adjusted
dftable.head()
dftable.to_csv(
    r"/Users/piyushnandwani/Documents/webscrapinggtransfermarkt.csv",
    index=False,
)

##############################################################################

Not relevant
Not relevant
Not relevant
Not relevant
Not relevant
